# OCR API 결과 시각화

## import

In [ ]:
import os, json
import requests

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

## 기본변수 선언

In [ ]:
IMG_PATH = 'img'
SECRET_PATH = '.secret'
RES_PATH = 'res'

## API 사용해보기

In [ ]:
secret_file = os.path.join(SECRET_PATH, 'key.json')

with open(secret_file) as f:
    secrets = json.loads(f.read())

if not os.path.isdir(RES_PATH): os.mkdir(RES_PATH)

api_url = secrets['API_URL']
secret_key = secrets['SECRET_KEY']

headers = {"secret": secret_key}

In [ ]:
file_list = os.listdir(IMG_PATH)
file_list_img = [file for file in file_list if file.split('.')[-1] in ['jpg', 'png', 'jpeg']]

for file_img in file_list_img:
    img_path = os.path.join(IMG_PATH, file_img)
    file_dict = {"file": open(img_path, "rb")}
    response = requests.post(api_url, headers=headers, files=file_dict).json()
    with open(os.path.join(RES_PATH, f'{file_img}_res.json'), 'w') as outfile:
        json.dump(response, outfile)

In [ ]:
fig, ax = plt.subplots(len(file_list_img), 1, figsize=(6, 6*len(file_list_img)))
ax = ax.flatten()
for i in range(len(ax)):
    img_path = os.path.join(IMG_PATH, file_list_img[i])
    img = Image.open(img_path, 'r')
    ocr_list = []
    ax[i].imshow(img)

    with open(os.path.join(RES_PATH, f'{file_list_img[i]}_res.json')) as f:
        img_patch = json.loads(f.read())
    
    # patch
    for patch in img_patch['ocr']['word']:
        x_pos = [x for x, _ in patch['points']]
        y_pos = [y for _, y in patch['points']]
        base_pos = min(x_pos), min(y_pos)
        width = max(x_pos) - min(x_pos)
        height = max(y_pos) - min(y_pos)
        rect = patches.Rectangle(base_pos, 
                            width, height, 
                            linewidth=1, edgecolor='r', facecolor='red', alpha=0.3)

        ax[i].add_patch(rect)
        ax[i].axis('off')
        ocr_list.append(patch['text'])